In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from torchvision.models import resnet18, ResNet18_Weights
import torch
import PIL

In [2]:
import numpy as np
from PIL import Image

DATASET_PATH = '/kaggle/input/dog-emotion/Dog Emotion/'

data = {'img': [], 'emotion': []}

for emotion in ['angry', 'happy', 'relaxed', 'sad']:
    for dirname, _, filenames in os.walk(DATASET_PATH + emotion + '/'):
        for filename in filenames:
            data['img'].append(os.path.join(dirname, filename))
            data['emotion'].append(emotion)

df = pd.DataFrame.from_dict(data)

In [4]:
df

,img,emotion
0,/kaggle/input/dog-emotion/Dog Emotion/angry/RD...,angry
1,/kaggle/input/dog-emotion/Dog Emotion/angry/GJ...,angry
2,/kaggle/input/dog-emotion/Dog Emotion/angry/GX...,angry
3,/kaggle/input/dog-emotion/Dog Emotion/angry/b3...,angry
4,/kaggle/input/dog-emotion/Dog Emotion/angry/Hf...,angry
...,...,...
3995,/kaggle/input/dog-emotion/Dog Emotion/sad/VNqe...,sad
3996,/kaggle/input/dog-emotion/Dog Emotion/sad/0Bnt...,sad
3997,/kaggle/input/dog-emotion/Dog Emotion/sad/aWoj...,sad
3998,/kaggle/input/dog-emotion/Dog Emotion/sad/8ihS...,sad


In [6]:
emotion_labels_map = {}

unique_emotions = df['emotion'].unique()
for i in range(len(unique_emotions)):
    emotion_labels_map[unique_emotions[i]] = i

In [8]:
df['emotion_label'] = df['emotion'].apply(lambda x: emotion_labels_map[x])
df

,img,emotion,emotion_label
0,/kaggle/input/dog-emotion/Dog Emotion/angry/RD...,angry,0
1,/kaggle/input/dog-emotion/Dog Emotion/angry/GJ...,angry,0
2,/kaggle/input/dog-emotion/Dog Emotion/angry/GX...,angry,0
3,/kaggle/input/dog-emotion/Dog Emotion/angry/b3...,angry,0
4,/kaggle/input/dog-emotion/Dog Emotion/angry/Hf...,angry,0
...,...,...,...
3995,/kaggle/input/dog-emotion/Dog Emotion/sad/VNqe...,sad,3
3996,/kaggle/input/dog-emotion/Dog Emotion/sad/0Bnt...,sad,3
3997,/kaggle/input/dog-emotion/Dog Emotion/sad/aWoj...,sad,3
3998,/kaggle/input/dog-emotion/Dog Emotion/sad/8ihS...,sad,3


In [19]:
from torch.utils.data import DataLoader, Dataset

class Dataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = PIL.Image.open(self.data.loc[idx, "img"]).convert('RGB')
        image = self.transform(image)
        label = torch.tensor(self.data.loc[idx, "emotion_label"])
        return image, label

In [20]:
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["img"], df["emotion_label"], random_state=42)

X_train.index = np.arange(len(X_train))
y_train.index = np.arange(len(y_train))
X_test.index = np.arange(len(X_test))
y_test.index = np.arange(len(y_test))

In [21]:
transform_test = v2.Compose([
    v2.Resize(size=(224, 224)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [22]:
train_ds = Dataset(pd.concat([X_train, y_train], axis=1), transform_test)
test_ds = Dataset(pd.concat([X_test, y_test], axis=1), transform_test)

train_loader = DataLoader(train_ds, batch_size=32, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=32, num_workers=4)

In [23]:
class Extraction:
    def __init__(self, network):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.network = network.eval().to(self.device)
        
    def extract(self, loader):
        data_tmp = []
        label_tmp = []

        with torch.no_grad():
            for x, y in loader:
                x = x.to(self.device)
            
                outputs = self.network(x)
                data_tmp.append(outputs.view(-1, 512).cpu().numpy())
                
                label_tmp.append(y.cpu().numpy())
                
        return np.vstack(data_tmp), np.hstack(label_tmp)

In [24]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

In [25]:
feature_extractor

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [26]:
ext = Extraction(feature_extractor)

train_feature, train_label = ext.extract(train_loader)
test_feature, test_label = ext.extract(test_loader)

In [27]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

param = {
    "kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    #'C': [0.1, 0.5, 1, 2, 5, 10]
}

svc = svm.SVC()
svm_grid = GridSearchCV(svc, param_grid=param, verbose=2, n_jobs=-1)

svm_grid.fit(train_feature, train_label)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=2)

In [28]:
y_pred_svm_grid = svm_grid.predict(test_feature)

In [29]:
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix

accuracy_score(test_label, y_pred_svm_grid)

0.734

[CV] END ......................................kernel=linear; total time=   3.4s
[CV] END ........................................kernel=poly; total time=   2.3s
[CV] END ........................................kernel=poly; total time=   2.8s
[CV] END .........................................kernel=rbf; total time=   3.1s
[CV] END .....................................kernel=sigmoid; total time=   2.6s
[CV] END ......................................kernel=linear; total time=   3.3s
[CV] END ........................................kernel=poly; total time=   2.9s
[CV] END ........................................kernel=poly; total time=   3.2s
[CV] END .........................................kernel=rbf; total time=   3.1s
[CV] END .....................................kernel=sigmoid; total time=   2.3s
[CV] END ......................................kernel=linear; total time=   3.1s
[CV] END ......................................kernel=linear; total time=   4.1s
[CV] END ...................